In [10]:
pip install transformers datasets torch evaluate sentencepiece

Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached datasets-4.4.1-py3-none-any.whl.metadata (19 kB)
  Using cached evaluate-0.4.6-py3-none-any.whl.metadata (9.5 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached safetensors-0.7.0-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached hf_xet-1.2.0-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.9 kB)
  Using cached dill-0.4.0-py3-none-any.whl.metadata (10 kB)
Using cached transformers-4.57.3-py3-none-any.whl (12.0 MB)
Using cached huggingface_hub-0.36.0-py3-none-any.whl (566 kB)
Using cached hf_xet-1.2.0-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86

In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Модель работает на устройстве: {device}")

Модель работает на устройстве: cuda


In [ ]:
#Блок обработки данных

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('AIgen_data.csv')
df

,id,text,is_human,lang
0,bba9c2379d3841a282f7231e7b9c8505,Car-free cities have become a subject of incre...,0,en
1,bba9c2379d3841a282f7231e7b9c8505,. Cost Savings: Car ownership and maintenance ...,0,en
2,bba9c2379d3841a282f7231e7b9c8505,. Implement Carpooling and Ride-Sharing: Promo...,0,en
3,31a9bb04ab214c1c8c4b8bb9f8ee87e6,"Car Free Cities Car-free cities, a concept gai...",0,en
4,31a9bb04ab214c1c8c4b8bb9f8ee87e6,. Parking lots and wide roads can be repurpose...,0,en
...,...,...,...,...
2515139,fcc125800bba4ba1b0bf0b6f00ddc4ac,. 4. Идеи подарков: отдайтесь тренду впечатлен...,0,ru
2515140,337db08bb3ab413093e7b756519128f0,Учёные сообщают о новых достижениях в области ...,0,ru
2515141,0ffe9b7f11574eb584969a0c9f6c2ccd,Представьте себе идеальный день: вы просыпаете...,0,ru
2515142,306168e83b444dc2827e02889a93494d,"Москва, 10 мая 2024 г. — В условиях растущей с...",0,ru


In [5]:
#Поиск всех записей на русском языке, оценка количества подходящих данных

df['lang'].unique()
len(df.loc[df['lang'] == 'ru'])

525982

In [ ]:
#Извлечение всех записей на русском языке подходящей длинны

df_ru = df.loc[(df['lang'] == 'ru') & (df['text'].str.len() < 512)]

In [6]:
from imblearn.under_sampling import RandomUnderSampler

#Ребаланс по целевой метке, с целью одинакового распределния классов
data = df_ru.drop(columns=['is_human'])
rebalance = df_ru['is_human']

rebalancer = RandomUnderSampler(sampling_strategy='majority', random_state = 2511)
data_rsmpl, rebalance_rsmpl = rebalancer.fit_resample(data, rebalance)

df_rebalanced = pd.DataFrame(data_rsmpl, columns=data.columns)
df_rebalanced['is_human'] = rebalance_rsmpl

print(len(df_rebalanced.loc[df['is_human'] == 1]))
print(len(df_rebalanced.loc[df['is_human'] == 0]))

36159
36159


In [7]:
#Изменение имён классов для дальнейшего удобства

df_rebalanced['is_human'] = 1 - df_rebalanced['is_human']

In [8]:
#Удаление лишних столбцов

df = df_rebalanced.drop(columns=['lang','id'])
df

,text,is_human
2419491,В Руанде заработала система доставки крови при...,1
2419496,После выхода фильма «Мушкетёры» актриса Мария ...,1
2419497,Она находится на стенде с чашкой чая; он застр...,1
2419502,Королю Англии Генриху IV удалось собрать около...,1
2419504,"Только к концу позднего обеда, за тем, что Арс...",1
...,...,...
2151466,Лучшие суши/роллы в городе! Заказываем уже бол...,0
2263952,"- А что будет, если прийти на выборы в майке с...",0
2215061,"Последний раз брала клубника подпорченная, вро...",0
2226454,"Вкусно, но не сравнить с помадкой от Красного ...",0


In [ ]:
#Блок обучения

In [ ]:
#Исходная модель

In [9]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset
import numpy as np
import evaluate

LOAD_PATH = ''
model_path = 'roberta'

tokenizer = AutoTokenizer.from_pretrained(model_path)
print("Токенизатор успешно загружен.")


model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)
print("Модель успешно загружена.")


dataset = Dataset.from_pandas(df)


dataset = dataset.train_test_split(test_size=0.2, seed=2511)


#Функция токенизации текста
def tokenize_function(examples):

    return tokenizer(
        examples["text"]
    )


tokenized_datasets = dataset.map(tokenize_function, batched=True)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


tokenized_datasets = tokenized_datasets.rename_column("is_human", "labels")


tokenized_datasets = tokenized_datasets.remove_columns(["text"])


tokenized_datasets.set_format("torch")


train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]


accuracy_metric = evaluate.load("accuracy")

#Функция для вычисления метрик в процессе обучения
def compute_metrics(eval_pred):
    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)


output_dir = "./xlm_roberta_ai_detector"

batch_size = 32
epochs = 100
steps_to_save = 10*epochs

#Инициализация параметров обучения
training_args = TrainingArguments(
    output_dir=output_dir,      
    learning_rate=2e-5,               
    per_device_train_batch_size=batch_size,    
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,               
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,               
    report_to="none",
    save_strategy="steps",        
    save_steps=steps_to_save,     
    save_total_limit=1,           
    load_best_model_at_end=True,
    eval_strategy="steps",
    eval_steps=steps_to_save
)

#Инициализация пайплайна обучения
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


print("--- Начинаем дообучение (fine-tuning) ---")
trainer.train()
print("--- Обучение завершено ---")


best_model_path = f"{output_dir}/best_model"
trainer.save_model(best_model_path)
print(f"Лучшая модель сохранена в: {best_model_path}")



print("\n--- Загрузка обученной модели для инференса ---")
trained_model = AutoModelForSequenceClassification.from_pretrained(best_model_path)
trained_tokenizer = AutoTokenizer.from_pretrained(best_model_path)


trained_model.eval()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trained_model.to(device)
print(f"Модель работает на устройстве: {device}")

#Получение результатов классификации
def get_ai_probability(text: str) -> float:

    inputs = trained_tokenizer(
        text,
        return_tensors="pt", 
        padding=True,
        truncation=True,
        max_length=128
    )


    inputs = {k: v.to(device) for k, v in inputs.items()}


    with torch.no_grad():
        outputs = trained_model(**inputs)
        logits = outputs.logits


    probabilities = torch.softmax(logits, dim=-1)


    ai_prob = probabilities[0, 1].item()

    return ai_prob



print("\n--- Тестирование модели ---")

test_text_human = "Я вчера ходил в кино с друзьями, фильм был неплохой."
test_text_ai = "Данная статья анализирует мультимодальные аспекты нейронных сетей."

prob_1 = get_ai_probability(test_text_human)
prob_2 = get_ai_probability(test_text_ai)

print(f"Текст: '{test_text_human}'")
print(f"Вероятность, что это ИИ: {prob_1:.4f} (ожидается низкая)")
print("-" * 20)
print(f"Текст: '{test_text_ai}'")
print(f"Вероятность, что это ИИ: {prob_2:.4f} (ожидается высокая)")

The tokenizer you are loading from 'roberta' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Токенизатор успешно загружен.
Модель успешно загружена.


Map:   0%|          | 0/57854 [00:00<?, ? examples/s]

Map:   0%|          | 0/14464 [00:00<?, ? examples/s]

/tmp/ipykernel_9706/3232971321.py:97: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


--- Начинаем дообучение (fine-tuning) ---


Step,Training Loss,Validation Loss


--- Обучение завершено ---
Лучшая модель сохранена в: ./xlm_roberta_ai_detector/best_model

--- Загрузка обученной модели для инференса ---


The tokenizer you are loading from './xlm_roberta_ai_detector/best_model' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Модель работает на устройстве: cuda

--- Тестирование модели ---
Текст: 'Я вчера ходил в кино с друзьями, фильм был неплохой.'
Вероятность, что это ИИ: 1.0000 (ожидается низкая)
--------------------
Текст: 'Данная статья анализирует мультимодальные аспекты нейронных сетей.'
Вероятность, что это ИИ: 1.0000 (ожидается высокая)


In [ ]:
#Модель с меньшим количеством эпох

In [9]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset
import numpy as np
import evaluate

LOAD_PATH = ''
model_path = 'roberta'

tokenizer = AutoTokenizer.from_pretrained(model_path)
print("Токенизатор успешно загружен.")


model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)
print("Модель успешно загружена.")


dataset = Dataset.from_pandas(df)


dataset = dataset.train_test_split(test_size=0.2, seed=2511)



def tokenize_function(examples):

    return tokenizer(
        examples["text"]
    )


tokenized_datasets = dataset.map(tokenize_function, batched=True)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


tokenized_datasets = tokenized_datasets.rename_column("is_human", "labels")


tokenized_datasets = tokenized_datasets.remove_columns(["text"])


tokenized_datasets.set_format("torch")


train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]


accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)


output_dir = "./xlm_roberta_ai_detector_less_epoch"

batch_size = 32
epochs = 30
steps_to_save = 10*epochs

training_args = TrainingArguments(
    output_dir=output_dir,      
    learning_rate=1e-5,               
    per_device_train_batch_size=batch_size,    
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,               
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,               
    report_to="none",
    save_strategy="steps",        
    save_steps=steps_to_save,     
    save_total_limit=1,           
    load_best_model_at_end=True,
    eval_strategy="steps",
    eval_steps=steps_to_save
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


print("--- Начинаем дообучение (fine-tuning) ---")
trainer.train()
print("--- Обучение завершено ---")


best_model_path = f"{output_dir}/best_model"
trainer.save_model(best_model_path)
print(f"Лучшая модель сохранена в: {best_model_path}")



print("\n--- Загрузка обученной модели для инференса ---")
trained_model = AutoModelForSequenceClassification.from_pretrained(best_model_path)
trained_tokenizer = AutoTokenizer.from_pretrained(best_model_path)


trained_model.eval()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trained_model.to(device)
print(f"Модель работает на устройстве: {device}")


def get_ai_probability(text: str) -> float:

    inputs = trained_tokenizer(
        text,
        return_tensors="pt", 
        padding=True,
        truncation=True,
        max_length=128
    )


    inputs = {k: v.to(device) for k, v in inputs.items()}


    with torch.no_grad():
        outputs = trained_model(**inputs)
        logits = outputs.logits


    probabilities = torch.softmax(logits, dim=-1)


    ai_prob = probabilities[0, 1].item()

    return ai_prob



print("\n--- Тестирование модели ---")

test_text_human = "Я вчера ходил в кино с друзьями, фильм был неплохой."
test_text_ai = "Данная статья анализирует мультимодальные аспекты нейронных сетей."

prob_1 = get_ai_probability(test_text_human)
prob_2 = get_ai_probability(test_text_ai)

print(f"Текст: '{test_text_human}'")
print(f"Вероятность, что это ИИ: {prob_1:.4f} (ожидается низкая)")
print("-" * 20)
print(f"Текст: '{test_text_ai}'")
print(f"Вероятность, что это ИИ: {prob_2:.4f} (ожидается высокая)")

The tokenizer you are loading from 'roberta' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Токенизатор успешно загружен.
Модель успешно загружена.


Map:   0%|          | 0/57854 [00:00<?, ? examples/s]

Map:   0%|          | 0/14464 [00:00<?, ? examples/s]

/tmp/ipykernel_624011/2197723979.py:97: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


--- Начинаем дообучение (fine-tuning) ---


Step,Training Loss,Validation Loss,Accuracy
300,0.317500,0.316735,0.891939
600,0.277900,0.258605,0.901410
900,0.268300,0.249421,0.907909
1200,0.255500,0.232281,0.912472
1500,0.239600,0.258539,0.904867
1800,0.241400,0.240450,0.907978
2100,0.217900,0.234037,0.917450
2400,0.192500,0.226263,0.915584
2700,0.194300,0.214611,0.921391
3000,0.211400,0.239875,0.916067


--- Обучение завершено ---
Лучшая модель сохранена в: ./xlm_roberta_ai_detector_less_epoch/best_model

--- Загрузка обученной модели для инференса ---


The tokenizer you are loading from './xlm_roberta_ai_detector_less_epoch/best_model' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Модель работает на устройстве: cuda

--- Тестирование модели ---
Текст: 'Я вчера ходил в кино с друзьями, фильм был неплохой.'
Вероятность, что это ИИ: 0.6899 (ожидается низкая)
--------------------
Текст: 'Данная статья анализирует мультимодальные аспекты нейронных сетей.'
Вероятность, что это ИИ: 0.9316 (ожидается высокая)


In [ ]:
#Обучение только классифицирующей головы

In [10]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset
import numpy as np
import evaluate

LOAD_PATH = ''
model_path = 'roberta'

tokenizer = AutoTokenizer.from_pretrained(model_path)
print("Токенизатор успешно загружен.")


model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)

for param in model.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = True

print("Модель успешно загружена.")


dataset = Dataset.from_pandas(df)


dataset = dataset.train_test_split(test_size=0.2, seed=2511)



def tokenize_function(examples):

    return tokenizer(
        examples["text"]
    )


tokenized_datasets = dataset.map(tokenize_function, batched=True)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


tokenized_datasets = tokenized_datasets.rename_column("is_human", "labels")


tokenized_datasets = tokenized_datasets.remove_columns(["text"])


tokenized_datasets.set_format("torch")


train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]


accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)


output_dir = "./xlm_roberta_ai_detector_only_head"

batch_size = 32
epochs = 100
steps_to_save = 10*epochs

training_args = TrainingArguments(
    output_dir=output_dir,      
    learning_rate=1e-5,               
    per_device_train_batch_size=batch_size,    
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,               
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,               
    report_to="none",
    save_strategy="steps",        
    save_steps=steps_to_save,     
    save_total_limit=1,           
    load_best_model_at_end=True,
    eval_strategy="steps",
    eval_steps=steps_to_save
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


print("--- Начинаем дообучение (fine-tuning) ---")
trainer.train()
print("--- Обучение завершено ---")


best_model_path = f"{output_dir}/best_model"
trainer.save_model(best_model_path)
print(f"Лучшая модель сохранена в: {best_model_path}")



print("\n--- Загрузка обученной модели для инференса ---")
trained_model = AutoModelForSequenceClassification.from_pretrained(best_model_path)
trained_tokenizer = AutoTokenizer.from_pretrained(best_model_path)


trained_model.eval()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trained_model.to(device)
print(f"Модель работает на устройстве: {device}")


def get_ai_probability(text: str) -> float:

    inputs = trained_tokenizer(
        text,
        return_tensors="pt", 
        padding=True,
        truncation=True,
        max_length=128
    )


    inputs = {k: v.to(device) for k, v in inputs.items()}


    with torch.no_grad():
        outputs = trained_model(**inputs)
        logits = outputs.logits


    probabilities = torch.softmax(logits, dim=-1)


    ai_prob = probabilities[0, 1].item()

    return ai_prob



print("\n--- Тестирование модели ---")

test_text_human = "Я вчера ходил в кино с друзьями, фильм был неплохой."
test_text_ai = "Данная статья анализирует мультимодальные аспекты нейронных сетей."

prob_1 = get_ai_probability(test_text_human)
prob_2 = get_ai_probability(test_text_ai)

print(f"Текст: '{test_text_human}'")
print(f"Вероятность, что это ИИ: {prob_1:.4f} (ожидается низкая)")
print("-" * 20)
print(f"Текст: '{test_text_ai}'")
print(f"Вероятность, что это ИИ: {prob_2:.4f} (ожидается высокая)")

The tokenizer you are loading from 'roberta' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Токенизатор успешно загружен.
Модель успешно загружена.


Map:   0%|          | 0/57854 [00:00<?, ? examples/s]

Map:   0%|          | 0/14464 [00:00<?, ? examples/s]

/tmp/ipykernel_624011/1991360434.py:104: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


--- Начинаем дообучение (fine-tuning) ---


Step,Training Loss,Validation Loss,Accuracy
1000,0.625100,0.629594,0.820174
2000,0.559100,0.565389,0.862832
3000,0.518500,0.504026,0.866012
4000,0.475000,0.459415,0.868847
5000,0.459700,0.426575,0.868501
6000,0.445000,0.404122,0.869607
7000,0.421400,0.387911,0.870506
8000,0.412000,0.376178,0.868985
9000,0.416300,0.366453,0.871958
10000,0.391600,0.359685,0.871197


--- Обучение завершено ---
Лучшая модель сохранена в: ./xlm_roberta_ai_detector_only_head/best_model

--- Загрузка обученной модели для инференса ---


The tokenizer you are loading from './xlm_roberta_ai_detector_only_head/best_model' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Модель работает на устройстве: cuda

--- Тестирование модели ---
Текст: 'Я вчера ходил в кино с друзьями, фильм был неплохой.'
Вероятность, что это ИИ: 0.3337 (ожидается низкая)
--------------------
Текст: 'Данная статья анализирует мультимодальные аспекты нейронных сетей.'
Вероятность, что это ИИ: 0.9645 (ожидается высокая)
